# Image Preprocessing

Importing Image Data Generator Library

In [1]:
pwd

'/home/wsuser/work'

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Rb_98YMyoN-zzee4ztUdxx8U4xexG4hGYrxQKq1GidT0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'arrythmiaclassification-donotdelete-pr-iyy1hnmipwjeum'
object_key = 'data - Copy.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO 
import zipfile 
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist() 
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

'data - Copy'/


In [5]:
from keras.preprocessing.image import ImageDataGenerator

Configure Image datagenerator class

In [6]:
#setting parameter for image data agumentationto the training data
train_datagen=ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#image data agumenatation to test dataset
test_datagen=ImageDataGenerator(rescale=1./225)

Applying ImageDataGenerator Functionality to train and test Dataset

In [7]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data - Copy/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data - Copy/test',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


# Model Building

Importing Libraries

In [8]:

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import keras

In [9]:
model= keras.Sequential()

Adding CNN Layers

In [10]:
#adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

Adding Dense Layers

In [11]:
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

Configuring the learning process

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Train the Model

In [13]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=20, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_1589/2925047980.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=20, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/20
480/480 [==============================] - 171s 355ms/step - loss: 0.6826 - accuracy: 0.7752 - val_loss: 0.6258 - val_accuracy: 0.8094
Epoch 2/20
480/480 [==============================] - 171s 355ms/step - loss: 0.2852 - accuracy: 0.9170 - val_loss: 0.5154 - val_accuracy: 0.8665
Epoch 3/20
480/480 [==============================] - 170s 354ms/step - loss: 0.2253 - accuracy: 0.9333 - val_loss: 0.3370 - val_accuracy: 0.9023
Epoch 4/20
480/480 [==============================] - 171s 356ms/step - loss: 0.1942 - accuracy: 0.9434 - val_loss: 0.3487 - val_accuracy: 0.8881
Epoch 5/20
480/480 [==============================] - 171s 356ms/step - loss: 0.1719 - accuracy: 0.9479 - val_loss: 0.3662 - val_accuracy: 0.8774
Epoch 6/20
480/480 [==============================] - 171s 356ms/step - loss: 0.1531 - accuracy: 0.9531 - val_loss: 0.3319 - val_accuracy: 0.8904
Epoch 7/20
480/480 [==============================] - 172s 358ms/step - loss: 0.1382 - accuracy: 0.9592 - val_loss: 0.3388 -

Saving the model

In [14]:
model.save('PREDICTION_MODEL.h5')

In [15]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 21.9 MB/s eta 0:00:01


In [16]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"VFECcIuGn6VBQtE-m_AjCrHtiyr7oMqmlk9Cc9xXG0_u"
}
client = APIClient(wml_credentials)

In [17]:
client

In [18]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space ['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [19]:
space_uid=guid_from_space_name(client,'arrythmia-prediction')
print("Space UID ="+space_uid)

Space UID =9c0d2961-8749-422a-91cc-7024e8706d70


In [20]:
client.set.default_space(space_uid)

'SUCCESS'

In [21]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [22]:
software_sapce_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [23]:
software_sapce_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [24]:
ls

'data - Copy'/   PREDICTION_MODEL.h5


In [25]:
!tar -zcvf arrythmia-classification.tgz PREDICTION_MODEL.h5

PREDICTION_MODEL.h5


In [26]:
model_details=client.repository.store_model(model='arrythmia-classification.tgz',meta_props={
    client.repository.ModelMetaNames.NAME: "arrythmia-classification",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_sapce_uid
})

In [27]:
model_id = client.repository.get_model_id(model_details)

In [28]:
model_id

'd1322879-d11e-401f-a207-57e6f9e4686c'

In [29]:
client.repository.download(model_id,'PREDICTION_MODEL.tar.gz')

Successfully saved model content to file: 'PREDICTION_MODEL.tar.gz'


'/home/wsuser/work/PREDICTION_MODEL.tar.gz'

In [30]:
ls

 arrythmia-classification.tgz   PREDICTION_MODEL.h5
'data - Copy'/                  PREDICTION_MODEL.tar.gz


Test the Model